<a href="https://colab.research.google.com/github/nak650228/ITEC/blob/20211010/VideoRestore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#◢ Video Restoration Prework

プリワークでは、Youtubeまたはファイル指定により動画ファイルをダウンロードし後後、DeepLearningによる補正を行う前処理を行います。各処理ごとの出力結果が、ユーザのGoogle Drive上のMovieフォルダに作られます。

その他にも以下の修正を施します。

　　・DeOldifyによるモノクロ画像のカラー化

　　・Deep Remasterによるノイズ等の除去

　　・アンシャープマスクの適用（OpenCVによる）

　　・Microsoft Bringing-Old-Photos-Back-to-LifeまたはGFPGANによる顔画像の修復  

　　・音声データの抽出し、動画修正時にそれを結合する

　　・オリジナル動画と画像処理後の動画を比較した動画ファイルを作成

　　・TecoGANを使った超高解像度化

　　・シーンの自動分割機能


今後の機能追加

　　・説明文に色や画像を貼って見た目を派手にする。

　　・超解像度化（4Kもしくは2Kに）

　　　　動画を1080pに変換し、最後に4Kなどで出力する機能の追加

　　　　RealESRGANまたは Waifuを適用する（アニメなど）

　　・DAINを使ったフレーム補完
         ToDo モノクロ動画の場合シーン分割がうまく動作しないなど不具合が

　　　　　あったため、この処理は最終処理の1段階前で実施し、その後DAINを使って
         
　　　　　フレーム補完を行う。

　　・ログの取得機能を実装する




#◢ GPUの確認

本処理には16GB以上のメモリを搭載したGPUが必要になります。

ランタイムに割り当てられたGPUを確認して、メモリ量が少ない場合は、ランタイムを出荷設定時にリセットします。16GB以上のモデルが割り当てられるまで、このリセットを繰り返してください。

In [1]:
#@title 割り当てられたGPUの確認
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla P100-PCIE-16GB, 460.32.03, 16280 MiB


#◢ 初期設定

In [2]:
#@title ライブラリ等のインストール
%cd /content
!pip install youtube_dl
!pip install ffmpeg
!pip install ffmpeg-python
#!pip install torchvision==0.5
!pip install torchvision
#!pip install torch==1.4
!pip install torch==1.9.0
#!pip install scipy==1.2.0
!pip install scipy
#!pip install imgaug==0.2.5
!pip install imgaug
#!pip install tensorflow==1.15.5
!pip install tensorflow

#シーン分割
!pip install scenedetect[opencv,progress_bar]

!pip install subprocess

import subprocess
import tensorflow as tf
import youtube_dl
import ffmpeg
import numpy as np
import imageio
import cv2
import torch
import glob
import shutil
import moviepy.editor as mpy
import os
import time
import datetime
from IPython.display import clear_output
from google.colab import files
torch.backends.cudnn.benchmark=True

clear_output()

IS_DEBUG=True

In [3]:
#@title Microsoft Bringing-Old-Photos-Back-to-Lifeのリポジトリーをクローンする
%cd /content
!git clone https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life.git photo_restoration

#@markdown Microsoft Bringing Old-Photos-Back-to-Lifeの学習済みモデルをダウンロード
# pull the syncBN repo
%cd photo_restoration/Face_Enhancement/models/networks
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../../

%cd Global/detection_models
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../

# download the landmark detection model
%cd Face_Detection/
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2
%cd ../

# download the pretrained model
%cd Face_Enhancement/
!wget https://facevc.blob.core.windows.net/zhanbo/old_photo/pretrain/Face_Enhancement/checkpoints.zip
!unzip -o checkpoints.zip
%cd ../

%cd Global/
!wget https://facevc.blob.core.windows.net/zhanbo/old_photo/pretrain/Global/checkpoints.zip
!unzip -o checkpoints.zip
%cd ../

! pip install -r requirements.txt

clear_output()

In [4]:
#@title DeOldifyの初期設定
%cd /content

!git clone https://github.com/jantic/DeOldify.git DeOldify
%cd DeOldify

#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

from os import path

!pip install -r colab_requirements.txt

import fastai
from deoldify.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth

colorizer = get_video_colorizer()

clear_output()

In [5]:
#@title ##**GFPGANをGithubからクローン**

# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
# Download the pre-trained model
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
clear_output()

In [6]:
#@title ##**Clone the repository of DeepRemaster** { display-mode: "form" }
%cd /content
!git clone https://github.com/satoshiiizuka/siggraphasia2019_remastering.git DeepRemaster
!cp -r /content/video.mp4 /content/DeepRemaster/
%cd /content/DeepRemaster

!wget --continue -O model/remasternet.pth.tar -- http://iizuka.cs.tsukuba.ac.jp/data/remasternet.pth.tar
clear_output()

#◢ 関数定義

In [10]:
#@title 各種関数の定義
#@markdown **decomposit_video(展開先フォルダ, ソースビデオファイル)  　　　　ビデオを静止画フレームに分解する**
#@markdown **unsharp_mask(ソースイメージ, 先鋭化の強さ)　　　　　　　　　　静止画のエッジなどを強調する**

def decomposit_video(video_folder,video_file, frame_rate):
  if os.path.isdir(video_folder):
    shutil.rmtree(video_folder)

  os.mkdir(video_folder)

  os.chdir(video_folder)

  decompose_command = 'ffmpeg -i ' + video_file + ' -vf fps=' + str(frame_rate) + ' %09d.png'
  subprocess.run(decompose_command, shell=True)
  #!ffmpeg -i /content/video.mp4 %09d.png

  #clear_output()

def composit_video(video_folder, video_file, frame_rate):
  if os.path.isfile(video_file):
    os.remove(video_file)

  compose_command='ffmpeg -f image2 -framerate ' + str(frame_rate) + ' -i ' + video_folder + '/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p ' + video_file
  subprocess.run(compose_command, shell=True)
  #clear_output()

def UnSharpMask(image_file, k):
    kernel = np.array([[-k/9.0, -k/9.0, -k/9.0],
                    [-k/9.0, 1 + (8 * k)/9.0, -k/9.0],
                    [-k/9.0, -k/9.0, -k/9.0]])
    dst = cv2.filter2D(image_file, -1, kernel)
    return dst

def make_sharp_kernel(k: int):
  return np.array([
    [-k / 9, -k / 9, -k / 9],
    [-k / 9, 1 + 8 * k / 9, k / 9],
    [-k / 9, -k / 9, -k / 9]
  ], np.float32)

def unsharp_mask_movie(video_folder, video_file, k, frame_rate):
  decomposit_video(video_folder,video_file, frame_rate)

  files = glob.glob(video_folder+'/*.png')
  for ifile in files:
    print("Now processing ",ifile)
    imageblur = cv2.imread(ifile, 0)
#    imagesharp = UnSharpMask(imageblur,k)

    kernel=make_sharp_kernel(1.0)
    imagesharp=cv2.filter2D(imageblur,-1,kernel).astype("uint8")
    cv2.imwrite(ifile, imagesharp)

  composit_video(video_folder, video_file, frame_rate)


from PIL import Image
import numpy as np
import torch
import torch.nn.functional as F
import torchvision

def load_tensor(image_file):
    with Image.open(image_file) as img:
        array = np.asarray(img, np.float32).transpose([2, 0, 1]) / 255.0
        tensor = torch.as_tensor(np.expand_dims(array, axis=0))  # rank 4
    return tensor


def sharpen_filter(image_file):
    kernel = np.array([[-2, -2, -2], [-2, 32, -2], [-2, -2, -2]], np.float32) / 16.0  # convolution filter
    with torch.no_grad():
        # [out_ch, in_ch, .., ..] : channel wiseに計算
        sharpen_k = torch.as_tensor(kernel.reshape(1, 1, 3, 3))

        color = load_tensor(image_file)  # color image [1, 3, H, W]
        # channel-wise conv(大事)　3x3 convなのでPadding=1を入れる
        multiband = [F.conv2d(color[:, i:i + 1,:,:], sharpen_k, padding=1) for i in range(3)]
        sharpened_image = torch.cat(multiband, dim=1)
        torchvision.utils.save_image(sharpened_image, image_file)


def sharpen_movie(video_folder, video_file, k, frame_rate):
  decomposit_video(video_folder,video_file, frame_rate)

  files = glob.glob(video_folder+'/*.png')
  for ifile in files:
    print("Now processing ",ifile)
    sharpen_filter(ifile)

  composit_video(video_folder, video_file, frame_rate)

def hist_filter(image_file):
    img = cv2.imread(image_file,0)

# create a CLAHE object (Arguments are optional).
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl1 = clahe.apply(img)

    cv2.imwrite(image_file,cl1)


def changehistgram_movie(video_folder, video_file, frame_rate):
  decomposit_video(video_folder,video_file, frame_rate)

  files = glob.glob(video_folder+'/*.png')
  for ifile in files:
    print("Now processing ",ifile)
    hist_filter(ifile)

  composit_video(video_folder, video_file, frame_rate)

def colorize_video(source_video, output_video, render_factor):
  print(source_video)
  print(output_video)

  if os.path.isfile("/content/DeOldify/video"):
    shutil.rmtree("/content/DeOldify/video")

  !mkdir -p '/content/DeOldify/video/source'

  command1="cp -r " + source_video + " /content/DeOldify/video/source/video.mp4"
  subprocess.run(command1,shell=True)
  print(command1)
#  !cp -r str(source_video) /content/DeOldify/video/source/video.mp4
  video_path = colorizer.colorize_from_file_name('/content/DeOldify/video/source/video.mp4', render_factor)
 
  command2="cp -r /content/DeOldify/video/result/video.mp4 " + output_video
  subprocess.run(command2,shell=True)
#  !cp -r /content/DeOldify/video/result/video.mp4 str(output_video)
  print(command2)
  if os.path.isfile("/content/DeOldify/video/result/video.mp4"):
    os.remove("/content/DeOldify/video/result/video.mp4")


def clear_logs(flag):
  if flag != True:
    clear_output()


!mkdir /content/reference



mkdir: cannot create directory ‘/content/reference’: File exists


#◢ 画像のダウンロード

In [9]:
#@title **Googleドライブの追加**
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print('Google Drive connected.')

Mounted at /content/drive
Google Drive connected.


In [33]:
#@title ##**プロジェクト名の入力** { display-mode: "form" }
##@markdown **プロジェクト名の入力**
projectname = 'RomanHoliday20211123\u20102' #@param {type:"string"}

In [34]:
#@title ##**ビデオのダウンロード** { display-mode: "form" }
#@markdown *ビデオへのリンク（YouTubeやTwitterなど）を入力するか、source_urlフィールドを空白にしてください（空白にした場合、コンピューターからビデオをアップロードするよう求められます）。*
#@markdown *プロジェクト名は任意でOKです。一応最後にGoogle Driveにプロジェクト名で指定したディレクトリが作られて、途中経過も含めて全ての動画ファイルがコピーされます*

import youtube_dl
import cv2
from IPython.display import clear_output

%cd /content

source_url = 'https://www.youtube.com/watch?v=K-iHVlAIphI&t=1s' #@param {type:"string"}

%cd /content
! rm -f /content/*.mp4

if source_url == '':
  dt_start_download = datetime.datetime.now()
  uploaded = files.upload()
  dt_end_download = datetime.datetime.now()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  file_name = "downloaded_video." + fn.split(".")[-1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    dt_start_download = datetime.datetime.now()
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    dt_end_download = datetime.datetime.now()
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    file_name = "downloaded_video." + fn.split(".")[-1]
    !mv -f $fn $file_name

!cp -r /content/downloaded_video.mp4 /content/video.mp4

clear_output()

fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
width_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_WIDTH))
height_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_HEIGHT))
print ("FPS of VIDEO: ",fps_of_video)
print ("Frames of VIDEO: ",frames_of_video)
print ("Width of VIDEO: ",width_of_video)
print ("Height of VIDEO: ",height_of_video)



FPS of VIDEO:  23
Frames of VIDEO:  5312
Width of VIDEO:  1280
Height of VIDEO:  720


In [35]:
#@title ##**動画の調整** { display-mode: "form" }

#@markdown *1分以上の動画をダウンロードすることはお勧めできません。また、タイトルに「スペース」や「ドット」が含まれている動画はアップロードしないでください。*
#@markdown *実行中にエラーが発生した場合は、このブロックを再度実行します。*
#@markdown *動画の長さを変更することができます。*

#@markdown **動画の長さを変更する場合は、その開始時間と終了時間を指定して下さい。**
target_start = '00:01:28' #@param {type:"string"}
target_end = '00:01:38' #@param {type:"string"}


if os.path.isfile("/content/cropped_video.mp4"):
    os.remove("/content/cropped_video.mp4")

!ffmpeg -i /content/downloaded_video.mp4  -ss $target_start -to $target_end /content/cropped_video.mp4

if os.path.isfile("/content/video.mp4"):
    os.remove("/content/video.mp4")

!cp /content/cropped_video.mp4 /content/video.mp4

#@markdown **モノクロ動画にAIで色を付ける場合は有効にしてください。**
is_deoldify = True #@param {type:"boolean"}


#@markdown **Deoldifyのレンダリングファクターを指定します。(お勧めは10～25）**
render_factor = 20  #@param {type: "slider", min: 5, max: 44}

#@markdown **ヒストグラムを調整する場合は有効にして下さい。**
is_hist = False #@param {type:"boolean"}

#@markdown **アンシャープマスクを適用する場合は有効にして下さい。**
is_unsharp = True #@param {type:"boolean"}
unsharp_ratio = '1.0' #@param {type:"string"}

#@markdown **DeepRemasterを有効にすると、低画質の動画からノイズなどを除去できます。**
is_DeepRemaster = True #@param {type:"boolean"}

#@markdown **DeepRemasterのカラー化機能を使うと、サンプル画像の色情報を転写できます。**
#@markdown **転写に使用するイメージ情報は/content/referenceディレクトリに置いてください。**
is_DeepRemaster_color = False #@param {type:"boolean"}


#@markdown **GANを使って、動画の顔部分を書き換えます。大体はがっかりしますが、時々きれいに修正されることがあります**

#@markdown **Microsoft Bringing-Old-Photos-Back-to-LifeまたはGFPGANを選択して、いずれかの手法で人物の精彩化を試みます。**
which_FaceGAN = 'Microsoft' #@param ["None", "GFPGAN", "Microsoft"] {allow-input: true}

#@markdown **フレーム補完利用の有無。**
#@markdown **フレームの数を倍にすることで、映像の動きをスムーズにします。**
is_DAIN = False #@param {type:"boolean"}


file_name="/content/cropped_video.mp4"
fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
width_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_WIDTH))
height_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_HEIGHT))

clear_output()

In [14]:
#@title ##**サイズ調整後の動画を表示（修復対象）** { display-mode: "form" }

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600, width=640))
else:
  files.download('/content/video.mp4')

print ("FPS of VIDEO: ",fps_of_video)
print ("Frames of VIDEO: ",frames_of_video)
print ("Width of VIDEO: ",width_of_video)
print ("Height of VIDEO: ",height_of_video)

FPS of VIDEO:  23
Frames of VIDEO:  240
Width of VIDEO:  1280
Height of VIDEO:  720


# ◢ 音声情報を抽出する

In [36]:
!ffmpeg -i /content/video.mp4 -vn -y -acodec copy output.aac
clear_logs(IS_DEBUG)

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

# ◢ 動画をアップスケールする

In [37]:
#@title ##**TecoGANによる動画のアップスケール（x2）** { display-mode: "form" }

if height_of_video <= 720: 

  %cd /content
  !git clone https://github.com/JoeyBallentine/Video-Inference.git
  !git clone https://github.com/skycrapers/TecoGAN-PyTorch.git

  %cd /content/TecoGAN-PyTorch/
  !bash /content/TecoGAN-PyTorch/scripts/download/download_models.sh BD TecoGAN
  !bash /content/TecoGAN-PyTorch/scripts/download/download_models.sh BI TecoGAN
  !cp /content/TecoGAN-PyTorch/pretrained_models/* /content/Video-Inference/models/


  %cd /content/Video-Inference/
  if os.path.isfile("./output/video.mp4"):
      os.remove("./output/video.mp4")

  !python run.py ./models/TecoGAN_BD_iter500000.pth --input "/content/video.mp4" --output "/content/Video-Inference/output/video.mp4"

  if (height_of_video * 4) >= 1080:
    !ffmpeg -i /content/Video-Inference/output/video.mp4 -vf scale=-1:1080 /content/upscaled_video.mp4
  else:
    !cp ./output/video.mp4 /content/upscaled_video.mp4

  !cp /content/upscaled_video.mp4 /content/video.mp4

  clear_logs(IS_DEBUG)

/content
fatal: destination path 'Video-Inference' already exists and is not an empty directory.
fatal: destination path 'TecoGAN-PyTorch' already exists and is not an empty directory.
/content/TecoGAN-PyTorch
/content/Video-Inference
100% (240 of 240) |#######################| Elapsed Time: 0:04:30 Time:  0:04:30
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --en

# ◢ コントラストを調整したり、動画にアンシャープマスクをかける

In [38]:
 #@title ##**動画のヒストグラムを調整する**
 if is_hist == True :
   changehistgram_movie("/content/datas", "/content/video.mp4", fps_of_video)
   !cp /content/video.mp4 /content/histgramed_video.mp4
clear_logs(IS_DEBUG)

In [39]:
#@title  ##**動画にアンシャープマスクをかける**
if is_unsharp == True:
  sharpen_movie("/content/datas", "/content/video.mp4", unsharp_ratio, fps_of_video)
  !cp /content/video.mp4 /content/sharpen_video.mp4
clear_logs(IS_DEBUG)

Now processing  /content/datas/000000065.png
Now processing  /content/datas/000000048.png
Now processing  /content/datas/000000016.png
Now processing  /content/datas/000000172.png
Now processing  /content/datas/000000024.png
Now processing  /content/datas/000000165.png
Now processing  /content/datas/000000032.png
Now processing  /content/datas/000000137.png
Now processing  /content/datas/000000138.png
Now processing  /content/datas/000000146.png
Now processing  /content/datas/000000128.png
Now processing  /content/datas/000000134.png
Now processing  /content/datas/000000057.png
Now processing  /content/datas/000000207.png
Now processing  /content/datas/000000053.png
Now processing  /content/datas/000000156.png
Now processing  /content/datas/000000002.png
Now processing  /content/datas/000000037.png
Now processing  /content/datas/000000101.png
Now processing  /content/datas/000000141.png
Now processing  /content/datas/000000130.png
Now processing  /content/datas/000000084.png
Now proces

In [40]:

compose_command='ffmpeg -f image2 -framerate ' + str(fps_of_video) + ' -i /content/datas/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/video.mp4'
subprocess.run(compose_command, shell=True)

CompletedProcess(args='ffmpeg -f image2 -framerate 23 -i /content/datas/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/video.mp4', returncode=1)

# #◢ モノクロ動画に色を付ける

In [41]:
#@title Deoldifyによるモノクロ動画のカラー化
if is_deoldify == True:

  %cd /content/DeOldify

  if os.path.isfile("/content/DeOldify/video"):
    shutil.rmtree("/content/DeOldify/video")

  !mkdir -p '/content/DeOldify/video/source'

  !cp -r /content/video.mp4 /content/DeOldify/video/source/video.mp4
  video_path = colorizer.colorize_from_file_name('/content/DeOldify/video/source/video.mp4', render_factor)
  !cp -r /content/DeOldify/video/result/video.mp4 /content/colorized_video.mp4
  !cp -r /content/colorized_video.mp4 /content/video.mp4
  if os.path.isfile("/content/DeOldify/video/result/video.mp4"):
    os.remove("/content/DeOldify/video/result/video.mp4")


#colorize_video('/content/video.mp4', '/content/colorized_video.mp4', render_factor)
#!cp /content/colorized_video /content/video.mp4
clear_logs(IS_DEBUG)

/content/DeOldify


Video created here: video/result/video.mp4


# ◢ ノイズ除去(Deep Remaster)

In [42]:
#@title ##**Remove frame noise** { display-mode: "form" }

if is_DeepRemaster == True:
  %cd /content/DeepRemaster

  if is_DeepRemaster_color == True:
    command = "python remaster.py --input /content/video.mp4 --refernce_dir /content/refernce --gpu --mindim " + str(int(height_of_video) * 2)
  else:
    command = "python remaster.py --input /content/video.mp4 --disable_colorization --gpu --mindim " + str(int(height_of_video) * 2)

  subprocess.run(command,shell=True)

  !cp /content/video.mp4 /content/denoised_video.mp4
#!python remaster.py --input /content/video.mp4 --disable_colorization --gpu --mindim 1080
#!python remaster.py --input /content/video.mp4 --disable_colorization --gpu
#clear_output()

clear_logs(IS_DEBUG)

/content/DeepRemaster


#◢ Microsoft Bringing-Old-Photos-Back-to-Lifeによる画像修正

In [43]:
#@title 動画ファイルを画像ファイルに分解
# ffmpeg extract - Generating individual frame PNGs from the source file.

if which_FaceGAN == "Microsoft":

  %cd /content/photo_restoration

  FRAME_INPUT_DIR = "/content/photo_restoration/input_frames"
  FRAME_OUTPUT_DIR = "/content/photo_restoration/output_frames"
  INPUT_FILEPATH = "/content/video.mp4"

  if os.path.isfile(FRAME_INPUT_DIR):
    shutil.rmtree(FRAME_INPUT_DIR)

  %shell mkdir -p '{FRAME_INPUT_DIR}'
  command = "ffmpeg -i " + INPUT_FILEPATH + " -vf framerate=" + str(fps_of_video) + " " +  FRAME_INPUT_DIR + "/%05d.png"
  subprocess.run(command,shell=True)

#  %shell ffmpeg -i '{INPUT_FILEPATH}' '{FRAME_INPUT_DIR}/%05d.png'

  if os.path.isfile(FRAME_OUTPUT_DIR):
    shutil.rmtree(FRAME_OUTPUT_DIR)
  
  %shell mkdir -p '{FRAME_OUTPUT_DIR}'


  png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
  from IPython.display import clear_output

  pngs_generated_count = int(png_generated_count_command_result.output.strip())


  #print(f"Input FPS: {fps}")
  print(f"{pngs_generated_count} frame PNGs generated.")

  # Checking if PNG do have alpha
  import subprocess as sp
  %cd {FRAME_INPUT_DIR}
  channels = sp.getoutput('identify -format %[channels] 00001.png')
  print (f"{channels} detected")

  # Removing alpha if detected
  if "a" in channels:
    print("Alpha detected and will be removed.")
    print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

  %cd /content/photo_restoration
  input_folder = FRAME_INPUT_DIR
  output_folder = FRAME_OUTPUT_DIR

  !rm -rf /content/photo_restoration/output_frames/*

  print (input_folder)
  print (output_folder)

  import os
  basepath = os.getcwd()
  #input_path = os.path.join(basepath, input_folder)
  #output_path = os.path.join(basepath, output_folder)
  #os.mkdir(output_path)
  #!rm -rf output_folder
  #os.mkdir(output_folder)

  !python run.py --HR --input_folder /content/photo_restoration/input_frames --output_folder /content/photo_restoration/output_frames --GPU 0

#create video

  %cd /content/photo_restoration/output_frames/final_output
  command="ffmpeg  -pattern_type glob -i '*.png' -c:v h264_nvenc -pix_fmt yuv420p -framerate " + str(fps_of_video) + " /content/beautified_video.mp4"
  #!ffmpeg  -pattern_type glob -i '*.png' -c:v h264_nvenc -pix_fmt yuv420p /content/beautified_video.mp4
  subprocess.run(command,shell=True)
  !rm -f /content/video.mp4
  !cp /content/beautified_video.mp4 /content/video.mp4
  #clear_output()

  clear_logs(IS_DEBUG)

/content/photo_restoration
230
230 frame PNGs generated.
/content/photo_restoration/input_frames
/bin/sh: 1: identify: not found detected
/content/photo_restoration
/content/photo_restoration/input_frames
/content/photo_restoration/output_frames
Running Stage 1: Overall restoration
Mapping: You are using the mapping model without global restoration.
Now you are processing 00001.png
Now you are processing 00002.png
Now you are processing 00003.png
Now you are processing 00004.png
Now you are processing 00005.png
Now you are processing 00006.png
Now you are processing 00007.png
Now you are processing 00008.png
Now you are processing 00009.png
Now you are processing 00010.png
Now you are processing 00011.png
Now you are processing 00012.png
Now you are processing 00013.png
Now you are processing 00014.png
Now you are processing 00015.png
Now you are processing 00016.png
Now you are processing 00017.png
Now you are processing 00018.png
Now you are processing 00019.png
Now you are processin

In [44]:
# Microsoft Bringing-Old-Photos-Back-to-Life では動画をデフォルトで25FPSで作ってしまうため、オリジナルのFPSに変換

decomposit_video("/content/datas","/content/beautified_video.mp4", 25)
  
composit_video("/content/datas","/content/adjusted_video.mp4",fps_of_video)

!cp /content/adjusted_video.mp4 /content/video.mp4
clear_logs(IS_DEBUG)

#◢ GFPGANによる顔画像の修復

In [45]:
#@title ##**GFPGANを使って動画ファイル中の顔画像を修復** { display-mode: "form" }

if which_FaceGAN == "GFPGAN":

  upload_folder = "/content/GFPGAN/inputs/upload"
  if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)

  os.mkdir(upload_folder)

  %cd /content/GFPGAN/inputs/upload

  !ffmpeg -i /content/video.mp4 %09d.png

# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions

  %cd /content/GFPGAN
  !rm -rf results
  !python inference_gfpgan.py --upscale 2 --test_path inputs/upload --save_root results --model_path experiments/pretrained_models/GFPGANCleanv1-NoCE-C2.pth --bg_upsampler realesrgan

  if os.path.isfile("/content/restored_video.mp4") :
    !rm -f /content/restored_video.mp4

  command="ffmpeg -f image2 -framerate " + str(fps_of_video) + " -i /content/GFPGAN/results/restored_imgs/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/beautified_video.mp4 "
  subprocess.run(command,shell=True)
  #!ffmpeg -f image2 -framerate 30 -i /content/GFPGAN/results/restored_imgs/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/beautified_video.mp4
  !rm -f /content/video.mp4
  !cp /content/beautified_video.mp4 /content/video.mp4

 
  clear_logs(IS_DEBUG)

# ◢ DAINによるフレーム補完


In [46]:
%cd /content
if is_DAIN == True:
  !git clone https://github.com/HeylonNHP/Dain-App.git
  !pip install ninja
 # !wget https://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
 # !dpkg -i cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb

  import os
  p = os.getenv('PATH')
  ld = os.getenv('LD_LIBRARY_PATH')
  os.environ['PATH'] = f"/usr/local/cuda-10.1/bin:{p}"
  os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-10.1/lib64:{ld}"


  %cd /content/Dain-App/my_package
  !sh build.sh
  %cd /content/Dain-App/PWCNet/correlation_package_pytorch1_0
  !sh build.sh

  !pip install PyQt5
  #!sudo add-apt-repository -y ppa:jonathonf/ffmpeg-4
  #!sudo apt update
  #!sudo apt -y install ffmpeg

  %cd /content/Dain-App

  ##!python  my_design.py -cli -h
  !cp /content/video.mp4 /content/Dain-App/video.mp4

  !python  my_design.py -cli --input "/content/Dain-App/video.mp4" -o "/content/Dain-App/example_folder/" -on "/content/interpolated_video.mp4" -m "model_weights/best.pth" -fh 2 --interpolations 2 --depth_awarenes 0 --loop 0 -p 0 --alpha 0 --check_scene_change 10 --png_compress 0 --crf 1 --pixel_upscale_downscale_before 1 --pixel_downscale_upscale_after 1 --pixel_upscale_after 1 --mute_ffmpeg 0 --split_size_x -1 --split_size_y -1 --split_pad 150 --half 0 --step_extract 1 --step_interpolate 1 --batch_size 1 --use_benchmark 0 --force_flow 1 --smooth_flow 0 --downscale -1 --fast_mode 0 -cif 1 -cof 1 -cc 1 -csc 1
  ##!python  my_design.py -cli --input "/content/Dain-App/video.mp4" -o "/content/Dain-App/example_folder/" -on "interpolated.mp4" -m "model_weights/best.pth" -fh 2 --interpolations 2 --depth_awarenes 0 --loop 0 -p 0 --alpha 0 --check_scene_change 10 --png_compress 0 --crf 1 --pixel_upscale_downscale_before 1 --pixel_downscale_upscale_after 1 --pixel_upscale_after 1 --mute_ffmpeg 0 --split_size_x -1 --split_size_y -1 --split_pad 150 --half 0 --step_extract 1 --step_interpolate 1 --batch_size 1 --use_benchmark 0 --force_flow 1 --smooth_flow 0 --downscale -1 --fast_mode 0
  !cp /content/Dain-App/example_folder/output_videos/*.mp4 /content/interpolated_video.mp4
  !cp /content/interpolated_video.mp4 /content/video.mp4
  clear_logs(IS_DEBUG)

/content


# ◢ 動画をシーンごとに分割する

In [47]:
#@title ##**動画をシーンごとに分割する** { display-mode: "form" }

splitted_folder="/content/scenes"
source_video="/content/video.mp4"
if os.path.isdir(splitted_folder):
    shutil.rmtree(splitted_folder)
os.mkdir(splitted_folder)

#シーンごとに動画を分割する
split_command="scenedetect -i " + source_video + " -o " + splitted_folder + " detect-content -t 27 list-scenes save-images split-video"
subprocess.run(split_command,shell=True)


clear_logs(IS_DEBUG)

#◢ 最終処理

In [48]:
%cd /content
!ffmpeg -i /content/video.mp4 -i /content/output.aac -c:v copy -c:a aac /content/final_output.mp4
#!cp /content/final_output.mp4 /content/video.mp4

clear_logs(IS_DEBUG)

/content
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --e

In [49]:
if is_DAIN != True:
  !ffmpeg -i /content/cropped_video.mp4 \
       -i /content/final_output.mp4 \
       -filter_complex "[0:v]scale=720:-2[v0];[1:v]scale=720:-2[v1];[v0][v1]hstack=inputs=2" \
       -vcodec libx264 -crf 23 /content/comparison_output.mp4

  clear_logs(IS_DEBUG)

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [50]:
#@title ファイルのバックアップ

ProjectDir="/content/drive/MyDrive/Movie/"+str(projectname)
print("ProjectDir")

if os.path.isfile(ProjectDir):
    shutil.rmtree(ProjectDir)

os.mkdir(ProjectDir)
os.chdir(ProjectDir)
!cp /content/*.mp4 .
!cp /content/output.aac .
!cp -rf /content/scenes .

clear_logs(IS_DEBUG)

ProjectDir
